In [1]:
# Imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow.keras as K
import tensorflow as tf

In [2]:
# Install tensorflow v 2.9.1 so model.save() works
if ((tf.__version__) != '2.9.1'):
    %pip install tensorflow==2.9.1
print("Tensorflow Version: " + (tf.__version__))

Tensorflow Version: 2.9.1


In [3]:
# Data Preprocessing
def preprocess_data(X, Y):
    """
    X: numpy.ndarray of shape (m, 32, 32, 3) containing the CIFAR10 data
    Y: numpy.ndarray of shape (m,) containing the CIFAR10 data labels
    Returns X_p and Y_p
    """
    X_p = K.applications.efficientnet.preprocess_input(X, data_format=None)
    Y_p = K.utils.to_categorical(Y, 10)

    return X_p, Y_p

In [4]:
# Prep variables for use in model building / training
MODEL_PATH = 'cifar10.h5'
optimizer = K.optimizers.Adam()
init = K.initializers.he_normal()

# Load CIFAR10 dataset
(x_train, y_train), (x_valid, y_valid) = K.datasets.cifar10.load_data()

# Pre-process the data
x_train_p, y_train_p = preprocess_data(x_train, y_train)
x_valid_p, y_valid_p = preprocess_data(x_valid, y_valid)

# input tensor
inputs = K.Input(shape=(32, 32, 3))

# Resize input
resize = K.layers.Lambda(
    lambda image: K.backend.resize_images(
    image, 240/32, 240/32, data_format='channels_last')
)(inputs)

In [5]:
# Load pretrained EfficientNetB1 Base Bodel
base_model = K.applications.EfficientNetB1(
    include_top=False,
    weights='imagenet',
    input_tensor=resize,
    input_shape=(240, 240, 3)
)
base_model.trainable = False

In [6]:
# Create Inception Block
def inception_block(A_prev, filters):
    """
    A_prev: output from the previous layer
    filters: tuple containing F1, F3R, F3, F5R, F5, FPP
        F1: number of filters in the 1x1 convolution
        F3R: number of filters in the 1x1 convolution before the 3x3
        convolution
        F3: number of filters in the 3x3 convolution
        F5R: number of filters in the 1x1 convolution before the 5x5
        convolution
        F5: number of filters in the 5x5 convolution
        FPP: number of filters in the 1x1 convolution after the max pooling
    All convolutions inside the inception block should use a ReLU activation
    Returns: the concatenated output of the inception block
    """
    F1 = filters[0]
    F3R = filters[1]
    F3 = filters[2]
    F5R = filters[3]
    F5 = filters[4]
    FPP = filters[5]

    conv_1x1 = K.layers.Conv2D(filters=F1,
                               kernel_size=(1, 1),
                               padding='same',
                               activation='relu'
                               )(A_prev)

    conv_1x1_3x3 = K.layers.Conv2D(filters=F3R,
                                   kernel_size=(1, 1),
                                   padding='same',
                                   activation='relu'
                                   )(A_prev)

    conv_3x3 = K.layers.Conv2D(filters=F3,
                               kernel_size=(3, 3),
                               padding='same',
                               activation='relu'
                               )(conv_1x1_3x3)

    conv_1x1_5x5 = K.layers.Conv2D(filters=F5R,
                                   kernel_size=(1, 1),
                                   padding='same',
                                   activation='relu'
                                   )(A_prev)

    conv_5x5 = K.layers.Conv2D(filters=F5,
                               kernel_size=(5, 5),
                               padding='same',
                               activation='relu'
                               )(conv_1x1_5x5)

    max_pool_3x3 = K.layers.MaxPooling2D(pool_size=(3, 3),
                                         strides=1,
                                         padding='same'
                                         )(A_prev)

    conv_1x1_pooled = K.layers.Conv2D(filters=FPP,
                                      kernel_size=(1, 1),
                                      padding='same',
                                      activation='relu'
                                      )(max_pool_3x3)

    output = K.layers.Concatenate()([
        conv_1x1, conv_3x3, conv_5x5, conv_1x1_pooled
    ])

    return output

In [7]:
# Add output layers
"""
Add a couple of inception blocks and MaxPooling2D
Referenced Deep CNN project for filter sizes and MaxPooling layers
"""
out = base_model.output
out = inception_block(out, [64, 96, 128, 16, 32, 32])
out = K.layers.MaxPooling2D(pool_size=(3, 3),
                            strides=2,
                            padding='same')(out)
out = inception_block(out, [64, 96, 128, 16, 32, 32])
out = K.layers.GlobalAveragePooling2D()(out)
out = K.layers.Dense(10, activation='softmax')(out)

In [8]:
# Compile Model
model = K.models.Model(inputs=inputs, outputs=out)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
# Train Model
model.fit(x=x_train_p,
          y=y_train_p,
          batch_size=64,
          epochs=5,
          validation_data=(x_valid_p, y_valid_p))

Epoch 1/5
782/782 [==============================] - 1530s 2s/step - loss: 0.6834 - accuracy: 0.7628 - val_loss: 0.4560 - val_accuracy: 0.8444
Epoch 2/5
782/782 [==============================] - 1526s 2s/step - loss: 0.4756 - accuracy: 0.8356 - val_loss: 0.4214 - val_accuracy: 0.8593
Epoch 3/5
782/782 [==============================] - 1526s 2s/step - loss: 0.4106 - accuracy: 0.8562 - val_loss: 0.4028 - val_accuracy: 0.8659
Epoch 4/5
782/782 [==============================] - 1525s 2s/step - loss: 0.3626 - accuracy: 0.8725 - val_loss: 0.4091 - val_accuracy: 0.8642
Epoch 5/5
782/782 [==============================] - 1559s 2s/step - loss: 0.3269 - accuracy: 0.8845 - val_loss: 0.3878 - val_accuracy: 0.8737


In [10]:
# Print Model Summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 lambda (Lambda)                (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['lambda[0][0]']                 
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                              

In [11]:
# Save Model
# model.save(MODEL_PATH)

In [12]:
# Load saved model and evaluate
load_model = K.models.load_model(MODEL_PATH)
load_model.evaluate(x_valid_p, y_valid_p, verbose=1)

313/313 [==============================] - 319s 1s/step - loss: 0.3723 - accuracy: 0.8761


[0.37231189012527466, 0.8761000037193298]